In [1]:
import sys
sys.version

'3.7.16 (default, Jan 17 2023, 22:20:44) \n[GCC 11.2.0]'

In [1]:
pip install snakemake

  Obtaining dependency information for snakemake from https://files.pythonhosted.org/packages/e0/a7/a13ec64b15cbd3dd0797847cb529e118e939d46968e80d0cdf7f5c638374/snakemake-9.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for appdirs from https://files.pythonhosted.org/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for immutables from https://files.pythonhosted.org/packages/ca/fc/20f4a5d5fcd4d17b46027d7d542190f5084f3d74abc6bdc2c0fab4d3deb3/immutables-0.21-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for configargparse from https://files.pythonhosted.org/packages/31/28/d28211d29bcc3620b1fece85a65ce5bb22f18670a03cd28ea4b75ede270c/configargparse-1.7.1-py3-none-any.whl.metadata
  Using cached connection_pool-0.0.3.tar.gz (3.8 kB)
  Preparing metadata (setup.py) ... done
 

In [7]:
pip install PuLP==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 2.8 MB/s eta 0:00:0000:0100:01
  Using cached pyparsing-3.1.4-py3-none-any.whl (104 kB)
  Attempting uninstall: PuLP
    Found existing installation: PuLP 3.1.1
    Uninstalling PuLP-3.1.1:
      Successfully uninstalled PuLP-3.1.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import snakemake

In [3]:
import os
os.getcwd()

'/home/dlrhksaud33'

In [4]:
os.chdir('/home/dlrhksaud33/snakemake')

In [5]:
os.getcwd()

'/home/dlrhksaud33/snakemake'

In [6]:
snakemake.__version__

'9.14.5'

In [32]:
%%writefile Snakefile
rule count:
    input:
        "{filename}.txt"
    output:
        "temp/wc_{filename}.txt"
    shell:
        "wc -w {input} > {output}"


Overwriting Snakefile


In [25]:
%%writefile Snakefile_2
rule concat:
    input:
        expand("temp/wc_{filename}.txt", filename=['A', 'B', 'C'])
    output:
        "temp/wc_all.csv"
    run:
        with open(output[0], "w") as out:
            out.write("filename,word_count\n")
            for infile in input:
                # wc 결과: "숫자 파일명"
                with open(infile) as f:
                    content = f.read().strip().split()
                    count = content[0]
                    filename = infile.split("/")[-1].replace("wc_", "").replace(".txt", "")
                    out.write(f"{filename},{count}\n")

Overwriting Snakefile_2


In [52]:
%%writefile Snakefile_trimming

rule trimming:
    input:
        fq = "data/{sample}.fq.gz"
    output:
        trim = "trimming/{sample}.trimmed.fq.gz"
    shell:
        "trim_galore --quality 20 --length 20 --gzip --path_to_cutadapt cutadapt --cores 40 {input.fq} -o {output.trim}"

Overwriting Snakefile_trimming


In [38]:
%%writefile Snakefile_HISAT2

rule hisat2:
    input:
        fq = "data/{sample}.fq.gz"
    output:
        sam = "sam/{sample}.sam"
    params:
        index = "ref/hisat2_index/reference_index_human"
    shell:
        """
        hisat2 -p 10 -x {params.index} -U {input.fq} -S {output.sam}
        """

Overwriting Snakefile_HISAT2


In [43]:
%%writefile Snakefile_sam_to_bam

rule sam_to_bam:
    input:
        sam = "sam/{sample}.sam"
    output:
        bam = "bam/{sample}.bam"
    shell:
        """
        samtools view -@ 10 -bS {input.sam} | samtools sort -@ 10 -o {output.bam}
        """

Writing Snakefile_sam_to_bam


In [47]:
%%writefile Snakefile_bam_index

rule bam_index:
    input:
        bam = "bam/{sample}.bam"
    output:
        bam_index = "bam_index/{sample}.bam.bai"
    shell:
        """
        samtools index {input.bam} -o {output.bam_index}
        """

Overwriting Snakefile_bam_index


In [46]:
%%writefile Snakefile_samtools
rule samtools:
    input:
        sam = "sam/{sample}.sam"
    output:
        bam = "bam/{sample}.bam"
    shell:
        """
        samtools view -@ 10 -bS {input.sam} | samtools sort -@ 10 -o {output.bam} | samtools index {output.bam}
        """


Overwriting Snakefile_samtools


In [49]:
%%writefile Snakefile_htseq

rule htseq:
    input:
        bam = "bam/{sample}.bam"
    output:
        counts = "htseq/{sample}.counts.txt"
    params:
        gtf = "gtf/Homo_sapiens.GRCh38.113_clean.gtf"
    shell:
        """
        htseq-count -f bam -r pos -s no -t exon -i gene_id --additional-attr gene_name -n 20 {input.bam} {params.gtf} > {output.counts}
        """

Overwriting Snakefile_htseq
